In [38]:
import pandas as pd
import textstat
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from sklearn.metrics import f1_score
import warnings
import numpy as np
import spacy
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import ast
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
sns.set_style("darkgrid")
from pprint import pprint
palette = "cool"


warnings.simplefilter(action='ignore')
# warnings.resetwarnings() to reset
nlp = spacy.load("en_core_web_sm") # spacy english model

In [30]:
df_train = pd.read_csv("cleaned_data.csv", index_col=0)
df_train.head()

,Text,Author,text_length,word_count,avg_word_length,type_to_token_ratio,punctuations_count,avg_sentence_length,POS_Nouns,POS_Verbs,flesch_reading_score,gunning_fog_index,unique_word_count,stopwords_number,cleaned_tokens
0,Scoring in PROC DISCRIM is as easy as validati...,AM,215,37,5.810811,0.840000,5,12.333333,0.305556,0.166667,58.99,10.33,21,12,"['scoring', 'proc', 'discrim', 'easy', 'valida..."
1,"In the GLM procedure, you may have used LSMEAN...",AM,782,129,6.062016,0.661972,11,18.428571,0.286822,0.093023,52.80,11.39,47,59,"['glm', 'procedure', 'may', 'used', 'lsmeans',..."
2,"The first problem, accuracy of the data file, ...",AM,990,159,6.226415,0.712766,19,16.000000,0.234177,0.158228,46.88,11.39,67,66,"['first', 'problem', 'accuracy', 'data', 'file..."
3,If the homogeneity of covariance matrices assu...,AM,934,146,6.397260,0.604938,17,18.250000,0.312925,0.081633,44.44,11.98,49,69,"['homogeneity', 'covariance', 'matrix', 'assum..."
4,"With a CONTRAST statement, you specify L, in t...",AM,1490,247,6.032389,0.488372,29,15.687500,0.282869,0.127490,55.54,9.15,63,122,"['contrast', 'statement', 'specify', 'l', 'cas..."


In [31]:
df_train = pd.read_csv("cleaned_data.csv", index_col=0)

In [32]:
tfidf_vectorizer = TfidfVectorizer()

X = tfidf_vectorizer.fit_transform(df_train['cleaned_tokens'])
y = df_train['Author']

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y, shuffle=True)

In [34]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_val_pred = model.predict(X_val)
report = classification_report(y_val, y_val_pred)
pprint(report)

('              precision    recall  f1-score   support\n'
 '\n'
 '          AM       1.00      0.38      0.56        13\n'
 '          CD       0.85      1.00      0.92        35\n'
 '          DM       0.88      0.97      0.92        29\n'
 '          DO       1.00      0.31      0.47        13\n'
 '          FE       0.93      0.93      0.93        28\n'
 '          TK       0.85      1.00      0.92        46\n'
 '\n'
 '    accuracy                           0.88       164\n'
 '   macro avg       0.92      0.76      0.79       164\n'
 'weighted avg       0.89      0.88      0.86       164\n')


In [35]:
feat_columns = ['text_length', 'unique_word_count', 'word_count', 'punctuations_count']
tfidf_features_addon = df_train.drop(columns=feat_columns)
tfidf_features_addon.head()

,Text,Author,avg_word_length,type_to_token_ratio,avg_sentence_length,POS_Nouns,POS_Verbs,flesch_reading_score,gunning_fog_index,stopwords_number,cleaned_tokens
0,Scoring in PROC DISCRIM is as easy as validati...,AM,5.810811,0.840000,12.333333,0.305556,0.166667,58.99,10.33,12,"['scoring', 'proc', 'discrim', 'easy', 'valida..."
1,"In the GLM procedure, you may have used LSMEAN...",AM,6.062016,0.661972,18.428571,0.286822,0.093023,52.80,11.39,59,"['glm', 'procedure', 'may', 'used', 'lsmeans',..."
2,"The first problem, accuracy of the data file, ...",AM,6.226415,0.712766,16.000000,0.234177,0.158228,46.88,11.39,66,"['first', 'problem', 'accuracy', 'data', 'file..."
3,If the homogeneity of covariance matrices assu...,AM,6.397260,0.604938,18.250000,0.312925,0.081633,44.44,11.98,69,"['homogeneity', 'covariance', 'matrix', 'assum..."
4,"With a CONTRAST statement, you specify L, in t...",AM,6.032389,0.488372,15.687500,0.282869,0.127490,55.54,9.15,122,"['contrast', 'statement', 'specify', 'l', 'cas..."


In [41]:
X_tfidf = tfidf_vectorizer.fit_transform(df_train['cleaned_tokens'])
numerical_features = tfidf_features_addon.drop(columns=["Author", "cleaned_tokens", "Text"])

scaler = MinMaxScaler()

numerical_features = numerical_features.apply(pd.to_numeric, errors='coerce')
numerical_features = numerical_features.fillna(0)
numerical_features_scaled = scaler.fit_transform(numerical_features)


X_combined = hstack([X_tfidf, numerical_features_scaled])
X_combined

# features are now combined, time to train the logisitc regression model

<816x5926 sparse matrix of type '<class 'numpy.float64'>'
	with 40980 stored elements in COOrdinate format>

In [42]:
X_train, X_val, y_train, y_val = train_test_split(X_combined, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_val_pred = model.predict(X_val)
report = f1_score(y_val, y_val_pred, average="weighted")
pprint(report)


0.910071621925959


In [43]:
df_test = pd.read_excel("Assignment_Data/Data_test.xlsx")
df_test.head()

,Text,Author
0,For time-dependent predictor variables (variab...,CD
1,The next task after settling upon a data acces...,DM
2,There are many techniques used to recode non-n...,DM
3,PROC AUTOREG can be used to illustrate the cal...,FE
4,A stratified sample is drawn from the non-INS ...,DM


In [44]:
from nltk.corpus import stopwords
import string
import re
from nltk.stem import WordNetLemmatizer
import spacy

nlp = spacy.load("en_core_web_sm") # spacy english model

def pos_proportions(text, pos_tag):
    doc = nlp(text)
    pos_counts = 0
    total_words = 0
    for token in doc:
        if token.is_alpha:
            total_words += 1
        if token.pos_ == pos_tag:
            pos_counts += 1
    return pos_counts / total_words if total_words > 0 else 0

def avg_word_length(sent): 
    sent_len = len(sent.split(" "))
    return sum([len(wrd)  for wrd in sent]) / sent_len

def text_tokenizer(sent):
    lemmatizer = WordNetLemmatizer()
    sent = sent.lower() # convert the text to lowercase
    tokens = re.split(r'\W+', sent) # split text based on non word characters
    clean_tokens = [lemmatizer.lemmatize(i) for i in tokens if i not in string.punctuation or not i.isdigit()] # removing number, punctuation and lemmatizing the text
    clean_tokens_no_stopwords = [i for i in clean_tokens if i not in stopwords.words('english')]
    return clean_tokens_no_stopwords

def calculate_ttr(text):
    tokens = text_tokenizer(text.lower())
    types = set(tokens)
    return len(types) / len(tokens) if tokens else 0

def creating_features():
    df_test['avg_word_length'] = df_test['Text'].apply(avg_word_length)
    df_test['type_to_token_ratio'] = df_test['Text'].apply(calculate_ttr)
    df_test['avg_sentence_length'] = df_test['Text'].apply(lambda x: np.mean([len(s) for s in str(x).split('.')]))
    df_test['POS_Nouns'] = df_test['Text'].apply(lambda x: pos_proportions(x, "NOUN"))
    df_test['POS_Verbs'] = df_test['Text'].apply(lambda x: pos_proportions(x, "VERB"))
    df_test['flesch_reading_score'] = df_test['Text'].apply(lambda x: textstat.flesch_reading_ease(x))
    df_test['gunning_fog_index'] = df_test['Text'].apply(lambda x: textstat.gunning_fog(x))
    df_test['stopwords_number'] = df_test['Text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords.words("english")]))
    return df_test

import contractions
from collections import Counter

remove_file_paths = lambda text: re.sub(r'\b[A-Za-z]:\\[^ ]*|\S+\.\S+', '', text).strip()

def text_tokenizer(sent):
    lemmatizer = WordNetLemmatizer()
    sent = remove_file_paths(sent) # removing all file names and file paths
    sent = sent.lower() # text to lowercase
    sent_expand = contractions.fix(sent) # expand the contractions (I'd => I would)
    tokens = re.split(r'\W+', sent_expand) # split based on non word characters
    no_stopwords = [i for i in tokens if i not in stopwords.words('english')]
    clean_tokens = [
        lemmatizer.lemmatize(i) if i.lower() != "sas" else "SAS"
        for i in no_stopwords 
        if i not in string.punctuation 
        and not any(char.isdigit() for char in i)  # Check for digits
        and i != ""
    ]# remove number, punctuation, remove words with numbers and lemmatize text
    return clean_tokens


def find_words_hyphen(text):
    """
    finding all the words with hyphen and to make them as compound words when tokenizing
    """
    pattern = re.compile(r'\b[A-Za-z]+(?:-[A-Za-z]+)+\b')
    return pattern.findall(text)

hyphenated_words = df_train['Text'].apply(find_words_hyphen).sum()
hyphenated_words_flat = [''.join(word) for word in hyphenated_words]
hyphenated_word_counts = Counter(hyphenated_words_flat)
hyphenated_word_list = list(hyphenated_word_counts)


df_test = creating_features()
df_test["cleaned_tokens"] = df_test["Text"].apply(text_tokenizer)
df_test.head()

,Text,Author,avg_word_length,type_to_token_ratio,avg_sentence_length,POS_Nouns,POS_Verbs,flesch_reading_score,gunning_fog_index,stopwords_number,cleaned_tokens
0,For time-dependent predictor variables (variab...,CD,6.440860,0.818182,149.000000,0.268817,0.139785,31.55,17.99,37,"[time, dependent, predictor, variable, variabl..."
1,The next task after settling upon a data acces...,DM,6.809524,0.836364,113.600000,0.357143,0.107143,33.24,16.50,29,"[next, task, settling, upon, data, access, met..."
2,There are many techniques used to recode non-n...,DM,7.256757,0.769231,88.666667,0.328767,0.136986,28.20,13.45,24,"[many, technique, used, recode, non, numeric, ..."
3,PROC AUTOREG can be used to illustrate the cal...,FE,6.000000,0.861111,90.750000,0.213115,0.147541,42.41,14.68,25,"[proc, autoreg, used, illustrate, calculation,..."
4,A stratified sample is drawn from the non-INS ...,DM,7.379310,0.909091,70.666667,0.241379,0.172414,44.71,10.78,9,"[stratified, sample, drawn, non, in, client, p..."


In [45]:
import pickle

with open('mwe_tokenizer.pkl', 'rb') as f:
    mwe_tokenizer = pickle.load(f)

def apply_mwe_to_tokens(row):
    tokens = mwe_tokenizer.tokenize(row)
    cleaned_tokens = [token.replace('\t', '') for token in tokens if not token.isnumeric()]
    return cleaned_tokens

df_test["cleaned_tokens"] = df_test["cleaned_tokens"].apply(apply_mwe_to_tokens)
df_test.head()

,Text,Author,avg_word_length,type_to_token_ratio,avg_sentence_length,POS_Nouns,POS_Verbs,flesch_reading_score,gunning_fog_index,stopwords_number,cleaned_tokens
0,For time-dependent predictor variables (variab...,CD,6.440860,0.818182,149.000000,0.268817,0.139785,31.55,17.99,37,"[time_dependent, predictor_variable, variable,..."
1,The next task after settling upon a data acces...,DM,6.809524,0.836364,113.600000,0.357143,0.107143,33.24,16.50,29,"[next, task, settling, upon, data, access, met..."
2,There are many techniques used to recode non-n...,DM,7.256757,0.769231,88.666667,0.328767,0.136986,28.20,13.45,24,"[many, technique, used, recode, non_numeric, i..."
3,PROC AUTOREG can be used to illustrate the cal...,FE,6.000000,0.861111,90.750000,0.213115,0.147541,42.41,14.68,25,"[proc, autoreg, used, illustrate, calculation,..."
4,A stratified sample is drawn from the non-INS ...,DM,7.379310,0.909091,70.666667,0.241379,0.172414,44.71,10.78,9,"[stratified, sample, drawn, non, in, client, p..."


In [48]:
# converting to tfidf vectors
X_tfidf = tfidf_vectorizer.transform(df_test['cleaned_tokens'].apply(lambda x: " ".join(x)))
numerical_features = df_test.drop(columns=["Author", "cleaned_tokens", "Text"])

scaler = MinMaxScaler()

numerical_features = numerical_features.apply(pd.to_numeric, errors='coerce')
numerical_features = numerical_features.fillna(0)
numerical_features_scaled = scaler.fit_transform(numerical_features)

# preparing the x_test and y_test data
X_test = hstack([X_tfidf, numerical_features_scaled])
y_test = df_test['Author']

<36x5926 sparse matrix of type '<class 'numpy.float64'>'
	with 1898 stored elements in COOrdinate format>

In [49]:
pred = model.predict(X_test)

pprint(classification_report(y_test, pred))

('              precision    recall  f1-score   support\n'
 '\n'
 '          AM       1.00      0.67      0.80         6\n'
 '          CD       0.93      1.00      0.97        14\n'
 '          DM       1.00      1.00      1.00         8\n'
 '          DO       1.00      0.50      0.67         2\n'
 '          FE       1.00      0.50      0.67         2\n'
 '          TK       0.57      1.00      0.73         4\n'
 '\n'
 '    accuracy                           0.89        36\n'
 '   macro avg       0.92      0.78      0.80        36\n'
 'weighted avg       0.93      0.89      0.89        36\n')
